In [1]:
from lib_framework.session_mgr import SessionMgr

# Loading the config this way to make it os independent on what you are running this on
import os
import sys
config_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2022_Street',
    'config.yml'
    )

sm = SessionMgr(config_file)

Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list14-4214-BrunnersMentalPrisoner.hashes. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list16-FL_kdIZUGpI.txt. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list17.txt. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list18.hash. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list19-paidanextra500000.hashes. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2022_Street/sample_hashes/list20-Authoritiesappeartohaveuncoveredavastnefariousconspiracy.hash

In [7]:
sm.load_main_pots(verbose=False, update_only=True)
sm.update_main_pots()

Number of new plains added to the JtR pot file: 0
Number of new plains added to the Hashcat pot file: 1499


In [8]:
sm.print_status()

Algorithm      :Total     :Cracked   :Remaining :Percentage
mysqlna        :4214      :10        :4204      :0%
half-md5       :2766      :0         :2766      :0%
raw-md5        :2933      :17        :2916      :1%
raw-sha1       :5455      :736       :4719      :13%
raw-sha256     :4997      :0         :4997      :0%
raw-sha384     :10004     :0         :10004     :0%
mssql05        :10812     :18        :10794     :0%
ssha           :2000      :1993      :7         :100%


In [9]:
sm.print_score()

Hash Type:     Value Per Crack:    Points Earned:      Total Possible Points:
mysqlna        17                  170                 71638
half-md5       3                   0                   8298
raw-md5        1                   17                  2933
raw-sha1       5                   3680                27275
raw-sha256     13                  0                   64961
raw-sha384     46                  0                   460184
mssql05        9                   162                 97308
ssha           5                   9965                10000

Total Score: 13994
Maximum Possible Score: 742597


In [5]:
sm.print_metadata_items("source")

source                   :Number of Hashes :Cracked
list14-BrunnersMentalPrisoner:4214             :10
list16-FL_kdIZUGpI       :2766             :0
list17                   :2933             :17
list18                   :5455             :187
list19-paidanextra500000 :4997             :0
list20-Authoritiesappearto:10004            :0
list21                   :10812            :18
list24                   :2000             :1043


In [6]:
sm.print_all_plaintext(meta_fields=['source'])

HASH TYPE: mysqlna ----------------------------------------------------------
source                   Plaintext
list14-BrunnersMentalPrisonerPERFECTION
list14-BrunnersMentalPrisonerFreeman
list14-BrunnersMentalPrisonermanmanman
list14-BrunnersMentalPrisonerDEVELOPMENT
list14-BrunnersMentalPrisonerTHETRUTH
list14-BrunnersMentalPrisonerTHANKYOU
list14-BrunnersMentalPrisonerHOMER
list14-BrunnersMentalPrisonerFreeman2
list14-BrunnersMentalPrisonerGOVERNMENT
list14-BrunnersMentalPrisonerfreemans
HASH TYPE: half-md5 ----------------------------------------------------------
source                   Plaintext
<No Cracked Hashes Exist For This Category>
HASH TYPE: raw-md5 ----------------------------------------------------------
source                   Plaintext
list17                   thegame!
list17                   areyou?
list17                   Joshua12
list17                   withyou.
list17                   don'tknow
list17                   don'tcare
list17                   St